In [ ]:
import Pkg
Pkg.add("Plots")

In [10]:

using Plots

In [1]:
function f1(x::Float64, α::Float64, β::Float64)
    return -2*α + β*(x^2-x)
end

function f1_solution(x::Float64)
    return x*(x-1)
end

function f2(x::Float64, α::Float64, β::Float64)
    return (α*π^2 + β)*sin(π*x)
end

function f2_solution(x::Float64)
    return sin(π*x)
end

f2_solution (generic function with 1 method)

In [2]:
function get_point(index::Int64, m::Int64)
    return convert(Float64, index/(m+1))     
end

function solution_on_points(α::Float64, β::Float64, m::Int64, f)
    exact_solution = zeros(m)
    for i in 1:m
        point = get_point(i, m)
        exact_solution[i] = f(point)::Float64
    end

    return exact_solution
end

solution_on_points (generic function with 1 method)

In [3]:
function solution_on_points(α::Float64, β::Float64, m::Int64, f)
    exact_solution = zeros(m)
    for i in 1:m
        point = get_point(i, m)
        exact_solution[i] = f(point)::Float64
    end

    return exact_solution
end

function solve_equation(α::Float64, β::Float64, m::Int64, f)

    h = 1/(m+1) 

    A = zeros((m, m))
    
    for i in 2:m-1
        A[i, i-1] = -α
        A[i, i  ] = β*h^2 + 2*α
        A[i, i+1] = -α
    end

    # linhas de contorno
    A[1, 1] = β*h^2 + 2*α
    A[1, 2] = -α

    A[m, m-1] = -α
    A[m, m] = β*h^2 + 2*α


    b = zeros(m)
    for i in 2:m-1
        
        point = get_point(i, m)
        b[i] = h^2*f(point, α, β)::Float64
    end

    #linhas de contorno
    b[1] = h^2*f(get_point(1, m), α, β)::Float64
    b[m] = h^2*f(get_point(m, m), α, β)::Float64

    num_solution = A\b

    return num_solution
end

solve_equation (generic function with 1 method)

Caso 1

In [6]:
α = 1.0
β = 1.0
m = 4

num_sol = solve_equation(α, β, m, f1)
exact_sol = solution_on_points(α, β, m, f1_solution)

display("Solução numérica: ")
display(num_sol)

display("Solução exata: ")
display(exact_sol)

"Solução numérica: "

4-element Vector{Float64}:
 -0.16
 -0.24000000000000002
 -0.23999999999999996
 -0.15999999999999998

"Solução exata: "

4-element Vector{Float64}:
 -0.16000000000000003
 -0.24
 -0.24
 -0.15999999999999998

Caso 2

In [7]:
α = 1.0
β = 1.0
m = 4

num_sol = solve_equation(α, β, m, f2)
exact_sol = solution_on_points(α, β, m, f2_solution)

display("Solução numérica: ")
display(num_sol)

display("Solução exata: ")
display(exact_sol)

"Solução numérica: "

4-element Vector{Float64}:
 0.6056405487528084
 0.9799469928471818
 0.9799469928471817
 0.6056405487528084

"Solução exata: "

4-element Vector{Float64}:
 0.5877852522924731
 0.9510565162951535
 0.9510565162951536
 0.5877852522924732